# Retrieval-Augmented Generation (RAG) Demo
This notebook shows how to combine document retrieval with LLM generation for more informed answers.

# Retrieval-Augmented Generation (RAG) Demo

In [ ]:
# Install required packages
%pip install faiss-cpu sentence-transformers openai

In [ ]:
import os, faiss, numpy as np, json
from sentence_transformers import SentenceTransformer
import openai

openai.api_key = os.getenv('OPENAI_API_KEY', None)
assert openai.api_key, 'Set OPENAI_API_KEY in your environment!'

In [ ]:
# Toy corpus
corpus = [
    'AI agents can reason and plan.',
    'Generative models create text and images.',
    'Retrieval augments LLMs with external knowledge.'
]
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(corpus)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

In [ ]:
# RAG answerer
def rag_answer(query):
    q_emb = model.encode([query])
    D, I = index.search(np.array(q_emb), k=1)
    context = corpus[I[0][0]]
    prompt = f"Context: {context}\nQuestion: {query}\nAnswer:"
    resp = openai.ChatCompletion.create(model='gpt-3.5-turbo', messages=[{'role':'user','content':prompt}])
    answer = resp.choices[0].message.content
    return context, answer

In [ ]:
# Run demo and save output
os.makedirs('./outputs', exist_ok=True)
query = 'How do agents use external knowledge?'
context, answer = rag_answer(query)
with open('./outputs/rag_output.json', 'w') as f:
    json.dump({'query': query, 'context': context, 'answer': answer}, f, indent=2)
print('Context:', context)
print('Answer:', answer)